In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
def create_model(image_shape):
    data_augmentation = keras.Sequential([
    layers.RandomFlip("horizontal",
                      input_shape=image_shape),
    layers.RandomRotation(0.1)])

    model = tf.keras.Sequential([
        data_augmentation,
        tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu'),
        tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu'),
        tf.keras.layers.MaxPooling2D(),
        tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'),
        tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'),
        tf.keras.layers.MaxPooling2D(),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax'),])

    model.compile(optimizer='adam',
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])
    return model

In [ ]:
directory = r'.\input_data'

class_names = ['yes', 'no']
classes_number = len(class_names)
batch_size = 64

img_height = 224
img_width = 224
img_bands = 3
image_shape = (img_height,img_width,img_bands)

epochs = 30

In [ ]:
training_data = tf.keras.preprocessing.image_dataset_from_directory(
    directory,
    class_names = class_names,
    color_mode='rgb',
    batch_size=batch_size,
    image_size=(img_height, img_width),
    shuffle=True,
    seed=123,
    validation_split=0.2,
    subset='training')

validation_data = tf.keras.preprocessing.image_dataset_from_directory(
    directory,
    class_names = class_names,
    color_mode='rgb',
    batch_size=batch_size,
    image_size=(img_height, img_width),
    shuffle=True,
    seed=123,
    validation_split=0.2,
    subset='validation')

training_class_names = training_data.class_names
validation_class_names = validation_data.class_names

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE
training_data = training_data.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
validation_data = validation_data.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
model = create_model(image_shape,classes_number)
model.fit(training_data, epochs=epochs, validation_data=validation_data)
test_loss, test_acc = model.evaluate(validation_data)